# Prototyping Heist in Julia

James Stapleton has written a nice python wrapper around gallery called Heist. Let's prototype such a system in Julia.

The basic operation of Heist is as follows:

1. Obtain a list of files. Heist has a nice function `heist.grab_art_files` that takes a directory name and a the start of a file name (prefix). It will make a list of all the root files with that prefix in the directory.
1. Make a `heist.ArtFile` object from the file list. The hame `ArtFile` is a misnomer and will be changed in future versions. A better name would be gallery or something like that. 
1. A "record spec" is created to specify a record to read. The object that is created is a python dictionary with the key being the "name" and the value of `heist.ArtRecordSpec`. The `heist.ArtRecordSpec` object needs the record type, the namespace, the module label, and the instance label. 
1. The records are added to the `heist.ArtFile` (curiously, one at a time).
1. The `artfile` can return an event loop. This is the `evt` object. 
1. The user does a `for` loop over the `evt` objct. 
1. Within the loop you can do `artfile.get_record`. You pass in the `record_spec`


In [2]:
# Setup IJulia
display(HTML("<style>.container { width:100% !important; }</style>"))  # Make full width

HTML{String}("<style>.container { width:100% !important; }</style>")

In [3]:
using Cxx, ROOT
using Lazy: @>

Let's deal with the files first. I want to use XrootD to stream. 

In [4]:
const filesDir = "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784"
const samDefinition = "gm2pro_mc_beamgun_1033_4000evts"

"gm2pro_mc_beamgun_1033_4000evts"

Let's just pick some of the files

In [5]:
files = readlines(`find $filesDir -name 'gm2*.30?*'`)

11-element Array{String,1}:
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root"
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.301.root"
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.302.root"
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.303.root"
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.304.root"
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.305.root"
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.306.root"
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.307.root"
 "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/15095757

We need to set up some C++ stuff

In [6]:
addHeaderDir(ENV["GALLERY_INC"], kind=C_System)
addHeaderDir(ENV["CANVAS_INC"], kind=C_System)
addHeaderDir(ENV["BOOST_INC"], kind=C_System)
addHeaderDir(ENV["CETLIB_INC"], kind=C_System)
addHeaderDir(ENV["CETLIB_EXCEPT_INC"], kind=C_System)
addHeaderDir(ENV["FHICLCPP_INC"], kind=C_System)
addHeaderDir(ENV["GM2RINGSIM_INC"], kind=C_System)
addHeaderDir(ENV["GM2DATAPRODUCTS_INC"], kind=C_System)
# Note that if you do `kind=C_User`, the #includes will hang

In [7]:
Libdl.dlopen("libgallery", Libdl.RTLD_GLOBAL)
Libdl.dlopen("libcanvas", Libdl.RTLD_GLOBAL)

Ptr{Void} @0x000000000b3f75e0

In [8]:
cxx"""
   #include "canvas/Utilities/InputTag.h"
   #include "gallery/Event.h"

   #include <vector>
"""

true

Make a list of files for Heist

In [9]:
struct HeistFiles
    files::Array{String,1}
    theVector::Cxx.CppValue
end

In [10]:
# Constructor that creates the vector
function HeistFiles(files::Array{String,1})
    fv = icxx"""std::vector<std::string> fv; return fv;"""
    for s ∈ files
        icxx"$fv.push_back($s);"
    end
    
    HeistFiles(files, fv)
end

HeistFiles

In [11]:
hf = HeistFiles(files);

Here's the include for the data product we care about

In [12]:
cxx"""
   #include "gm2dataproducts/mc/actions/track/TrackingActionArtRecord.hh"
"""

true

Now, we want to make the record specs. We won't use a dictionary here.

In [13]:
struct HeistRecordSpec
   productName::String
   artInputTag::Cxx.CppValue

   function HeistRecordSpec(; namespace="", productType="", moduleLabel="", instanceLabel="")
      @assert productType != ""
      @assert moduleLabel != ""
      productName = namespace == "" ? productType : namespace * "::" * productType
      inputTagString = instanceLabel == "" ? moduleLabel : moduleLabel * ":" * instanceLabel
      new(productName, icxx"""art::InputTag it($inputTagString); return it;""")
   end
end

In [14]:
trackingActionSpec = HeistRecordSpec(
                           namespace = "gm2truth",
                           productType = "TrackingActionArtRecordCollection",
                           moduleLabel = "artg4")

HeistRecordSpec("gm2truth::TrackingActionArtRecordCollection", (class art::InputTag) {
}
)

In [15]:
icxx"std::cout << $(trackingActionSpec.artInputTag).encode() << std::endl;"

artg4


(class std::basic_ostream<char> &) {
}


In [16]:
trackingActionSpec.productName

"gm2truth::TrackingActionArtRecordCollection"

We now have enough to create a gallery event looper

In [17]:
icxx"std::cout << $(hf.theVector)[10] << std::endl;"

/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.30.root


(class std::basic_ostream<char> &) {
}


NOTE: This is not the best way. See a better way below

Remember how `for` loops work...

```julia
for i in iter   # or  "for i = iter"
    # body
end
```
is translated into:

```julia
state = start(iter)
while !done(iter, state)
    (i, state) = next(iter, state)
    # body
end
```

So we need to define the iterator object, state object and `start`, `done`, and `next`

In [18]:
struct HeistLoop
   hf::HeistFiles
   count::Int
end

In [19]:
Base.start(::HeistLoop) = icxx"""gallery::Event ev($(hf.theVector)); return ev;"""

In [20]:
function Base.next(HL::HeistLoop, state)
    @cxx state->next()
    return (state, state)
end

In [21]:
function Base.done(HL::HeistLoop, state)
    e = @cxx state->eventEntry()
    atEnd = @cxx state->atEnd()
    return atEnd || e ≥ HL.count  
end

In [22]:
Base.eltype(::Type{HeistLoop}) = Cxx.CppValue

Let's see if this works

In [23]:
h = HeistLoop(hf, 10)

HeistLoop(HeistFiles(String["/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.301.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.302.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.303.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.304.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.305.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.306.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.307.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/150957578

In [24]:
s = start(h)

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root


(class gallery::Event) {
}


In [25]:
done(h, s)

false

In [26]:
(i, s) = next(h, s)

((class gallery::Event) {
}
, (class gallery::Event) {
}
)

In [27]:
@cxx s->eventEntry()

1

In [28]:
done(h, s)

false

In [29]:
for i ∈ HeistLoop(hf, 30)
    println( @cxx i->eventEntry() )
end

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [30]:
h = HeistLoop(hf, 30)

HeistLoop(HeistFiles(String["/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.301.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.302.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.303.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.304.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.305.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.306.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.307.root", "/pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/150957578

In [31]:
s = start(h)

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root


(class gallery::Event) {
}


In [32]:
s

(class gallery::Event) {
}


In [33]:
@cxx s->numberOfEventsInFile()

4000

Problems with this...
* We miss the first event
* The count is by event number, not by # of events seen

Maybe make the state a struct with count and event and see if we can make it incomplete at start (not have the event at the beginning). It will work if `state` is mutable. See 
https://docs.julialang.org/en/stable/manual/constructors/#Incomplete-Initialization-1

Change the name of HeistLoop to HeistEvents

Ok - let's try this ...

In [34]:
a = nothing ; typeof(a)

Void

In [35]:
# Define a HeistEvent
mutable struct HeistEvent
   galleryEvent::Cxx.CppValue
   nSeen::Int

   HeistEvent(hf::HeistFiles) = new( icxx"gallery::Event ev($(hf.theVector)); return ev;", 1)
end

nextEvent(he::HeistEvent) = ( @cxx he.galleryEvent->next() ; he.nSeen += 1; he )
atEnd(he::HeistEvent) = @cxx he.galleryEvent->atEnd()


atEnd (generic function with 1 method)

In [36]:
he = HeistEvent(hf)

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root


HeistEvent((class gallery::Event) {
}
, 1)

In [37]:
atEnd(he)

false

In [38]:
@cxx he.galleryEvent->eventEntry()

0

In [39]:
# Define a looper
struct HeistEvents
   hf::HeistFiles
   nDesired::Int
end

In [40]:
Base.start(::HeistEvents) = nothing   # Don't do anything at the start -- next will make the HeistEvent -- this allows us to see the first event

Base.next(hes::HeistEvents, state::Void) = (he = HeistEvent(hes.hf) ; (he, he) )  # Multiple dispatch in action!
Base.next(hes::HeistEvents, state::HeistEvent) = (nextEvent(state), state)

Base.done(hes::HeistEvents, state::Void) = false
Base.done(hes::HeistEvents, state::HeistEvent) = atEnd(state) || state.nSeen ≥ hes.nDesired

In [41]:
hes = HeistEvents(hf, 7);

In [42]:
s = start(hes)

In [43]:
typeof(s)

Void

In [44]:
(i, s) = next(hes, s)

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root


(HeistEvent((class gallery::Event) {
}
, 1), HeistEvent((class gallery::Event) {
}
, 1))

In [45]:
@cxx i.galleryEvent->eventEntry()

0

In [46]:
i.nSeen

1

In [47]:
(i, s) = next(hes, s)

(HeistEvent((class gallery::Event) {
}
, 2), HeistEvent((class gallery::Event) {
}
, 2))

In [48]:
done(hes, s)

false

In [49]:
@time for i ∈ HeistEvents(hf, 10)
    println( @cxx i.galleryEvent->eventEntry() )
end

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root
0
1
2
3
4
5
6
7
8
9
  0.115149 seconds (316 allocations: 8.141 KiB)


Ok - now we need to get some data out of this. Let's get an event

In [75]:
he = HeistEvent(hf);

Successfully opened file /pnfs/GM2/mc/commission_mdc2_1033/runs_1509575000/1509575784/gm2ringsim_muon_beamgun_truth_1509575784.300.root


Let's define a `getRecord` method for `HeistEvent`

In [118]:
function getRecord(he::HeistEvent, rs::HeistRecordSpec)

    pn = rs.productName
    it = rs.artInputTag
    ev = he.galleryEvent
    cxxparse("#define TYP " * pn)
    icxx"""
        auto const h = $ev.getValidHandle<TYP>($it); return h;
    """
end

getRecord (generic function with 1 method)

In [119]:
trackingActionSpec

HeistRecordSpec("gm2truth::TrackingActionArtRecordCollection", (class art::InputTag) {
}
)

In [121]:
th = getRecord(he, trackingActionSpec)

(const class gallery::ValidHandle<class std::vector<struct gm2truth::TrackingActionArtRecord, class std::allocator<struct gm2truth::TrackingActionArtRecord> > >) {
}


In [122]:
icxx"$th->size();"

0x0000000000000255

In [123]:
?Pkg.generate

No documentation found.

`Base.Pkg.generate` is a `Function`.

```
# 1 method for generic function "generate":
generate(pkg, license) in Base.Pkg at pkg/pkg.jl:307
```


In [125]:
Pkg.add("PkgDev")

INFO: Cloning cache of PkgDev from https://github.com/JuliaLang/PkgDev.jl.git
INFO: Installing PkgDev v0.1.6
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of PkgDev
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [127]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating cache of NaNMath...
INFO: Updating cache of Compat...
INFO: Updating cache of JSON...
INFO: Computing changes...
INFO: Upgrading NaNMath: v0.2.6 => v0.3.0


In [129]:
Pkg.add("Documenter")

INFO: Cloning cache of DocStringExtensions from https://github.com/JuliaDocs/DocStringExtensions.jl.git
INFO: Cloning cache of Documenter from https://github.com/JuliaDocs/Documenter.jl.git
INFO: Installing DocStringExtensions v0.4.1
INFO: Installing Documenter v0.12.3
INFO: Package database updated


In [130]:
LOAD_PATH

2-element Array{Any,1}:
 "/Users/lyon/Development/julia/julia-centos6-prod8/julia/usr/local/share/julia/site/v0.6"
 "/Users/lyon/Development/julia/julia-centos6-prod8/julia/usr/share/julia/site/v0.6"      

In [131]:
import PkgDev

INFO: Precompiling module PkgDev.


In [ ]:
PkgDev.config()

PkgDev.jl configuration:
Enter user name:Enter user email:Enter GitHub user [rm -f ./rik_kernel-8129371a-a106-4d67-856d-90baa8961a76.json]:

In [ ]:
cxxparse(#include "gm2dataproducts/mc/actions/track/TrackingActionArtRecord.hh")